Import necessary lib.

In [1]:
import re

First check how many records in text.data.jsonl.

In [2]:
with open('text.data.jsonl','r') as fin:
    total_count = 0
    for line in fin:
        match = re.match('\{"id": ', line)
        if match:
            total_count+=1
    print("Total record count: " + str(total_count))

Total record count: 183149


In [3]:
with open('text.data.jsonl','r') as fin:
    no_opinion_count = 0
    for line in fin:
        match = re.search('"opinions": \[\]', line)
        if match:
            no_opinion_count += 1
    print("Records without opinions count: " + str(no_opinion_count))
    print("Records with opinions count: " + str(total_count - no_opinion_count))

Records without opinions count: 216
Records with opinions count: 182933


Now extract record id and opinions sections into a new file (opinions.txt).

In [4]:
def re_repl_count(pattern,replacement_texts,line):
    count = 0
    tmp = re.findall(pattern,line)
    count = len(tmp)
    result_str = re.sub(pattern,replacement_texts,line) 
    return result_str, count;

In [17]:
with open('opinions.csv','w') as fout:
    with open('text.data.jsonl','r') as fin:
        count_list = [0 for i in range(20)]
        count_sub = 0
        for line in fin: 
            if re.match(r'\{"id": ([0-9]+), .+\n', line):
                i = 0
                #get a record id
                record_id = re.sub(r'\{"id": ([0-9]+), "name": .+\n', r'"\1"',line)
                
                #sub 1: extract opinions segment and store it in result
                result, count = re_repl_count(r'.+(?="opinions": )"opinions": \[(.*?)\](?=, "parties"), "parties": .+\n', r'\1\n', line)
                count_list[i] += count
                i = i + 1
                
                #sub 2:
                result, count = re_repl_count(r' null(?=\})', r' "null"', result)
                count_list[i] += count
                i = i + 1
                
                #sub 3: take the above result and add a newline and record id at the beginning of each type-text-author triplet.
                result, count = re_repl_count(r'\{"type": (".*?")(?=, "text": ), "text": (".*?")(?=, "author": ), "author": (".*?")\}(, )?', record_id + r'\t\1\t\2\t\3', result) 
                count_list[i] += count
                i = i + 1
                
                fout.write(result)
                count_sub = i
                
        for j in range(count_sub): 
            print('substitution ' + str(j + 1) + ' count: ' + str(count_list[j])) 

substitution 1 count: 183149
substitution 2 count: 1881
substitution 3 count: 194365


Side notes:
One record can have multiple type-text-author triplets.
In the extracted opinions.txt, there are 194365 type-text-author triplets that cover 182933 records. Each triplet is preceeded with the corresponding record id separated by a tab.

In [18]:
with open('without_opinions.csv','w') as fout:
    with open('text.data.jsonl','r') as fin:
        count_list = [0 for i in range(20)]
        count_sub = 0
        for line in fin: 
            if re.match(r'\{"id": ([0-9]+), .+\n', line):
                i = 0
                #get a record id
                record_id = re.sub(r'\{"id": ([0-9]+), "name": .+\n', r'"\1"',line)
                
                #sub 1: extract non-opinions segment and store it in result
                result, count = re_repl_count(r'\{"id": [0-9]+, ("name": ".*?)(?=, "data": ), "data": \{"opinions": .*?(?=, "parties": )(, "parties": .+)', r'\1\2', line)
                count_list[i] += count
                i = i + 1
                
                #sub 2:
                """
                1 - name
                2 - name_abbreviation
                3 - decision_date
                4 - docket_number
                5 - first_page
                6 - last_page
                7 - citations
                8 - the rest
                """
                result, count = re_repl_count(r'"name": (".*?")(?=, "name_abbreviation": ), "name_abbreviation": (.*?)(?=, "decision_date": ), "decision_date": ("[^"]*?"), "docket_number": ("[^"]*?"), "first_page": ("[^"]*?"), "last_page": ("[^"]*?"), ("citations": \[.*?\], )(?="volume": )("volume": .+\n)', r'\1\t\2\t\3\t\4\t\5\t\6\t\7\t\8', result)
                count_list[i] += count
                i = i + 1
                
                #sub 3: citations
                if re.search(r'"cite": "[^"]*?"., ."type": "', result):
                    result, count = re_repl_count(r'(.*?)(?="citations": )"citations": \[\{"type": ("[^"]*?"), "cite": ("[^"]*?")\}, \{"type": ("[^"]*?"), "cite": ("[^"]*?")\}\](.+\n)', r'\1\t\2\t\3\t\4\t\5\t\6', result)
                    count_list[i] += count
                #sub 4: citations
                else:    
                    result, count = re_repl_count(r'(.*?)(?="citations": )"citations": \[\{"type": ("[^"]*?"), "cite": ("[^"]*?")\}\](.+\n)', r'\1\t\2\t\3\t\t\t\4', result)
                    count_list[i+1] += count
                i = i + 2
                
                #sub 5:
                """
                1 - the rest
                2 - volume_number
                3 - full_name
                4 - the rest
                """                
                result, count = re_repl_count(r'(.*?)(?="volume": )"volume": \{"volume_number": ("[^"]*?")\}, "reporter": \{"full_name": (".*?")\}, ("court": .+\n)', r'\1\2\t\3\t\4', result)                
                count_list[i] += count
                i = i + 1
                
                #sub 6:
                """
                1 - the rest
                (court)
                2 - id
                3 - name
                4 - name_abbreviation
                5 - jurisdiction_url
                6 - slug
                7 - the rest
                """                
                result, count = re_repl_count(r'(.*?)(?="court": )"court": ."id": ([^,]*?), "name": (".*?")(?=, "name_abbreviation": ), "name_abbreviation": (".*?")(?=, "jurisdiction_url": ), "jurisdiction_url": ([^,]*?), "slug": ("[^"]*?")., ("jurisdiction": .+\n)', r'\1\2\t\3\t\4\t"\5"\t\6\t\7', result)                
                count_list[i] += count
                i = i + 1                  
                
                #sub 7:
                """
                1 - the rest
                (jurisdiction)
                2 - id
                3 - slug
                4 - name
                5 - name_long
                6 - whitelisted
                7 - the rest
                """  
                result, count = re_repl_count(r'(.*?)(?="jurisdiction": )"jurisdiction": \{"id": ([0-9]*?), "slug": ("[^"]*?"), "name": ("[^"]*?"), "name_long": ("[^"]*?"), "whitelisted": ([^"]*?)\}, ("casebody": .+\n)', r'\1\t"\2"\t\3\t\4\t\5\t"\6"\t\7', result)
                count_list[i] += count
                i = i + 1
                
                #sub 8:
                """
                1 - the rest
                (casebody)
                2 - status
                3 - parties
                4 - attorneys
                5 - head_matter
                6 - judges
                """  
                result, count = re_repl_count(r'(.+)\t"casebody": \{"status": ("[a-z]+"), "parties": \[(.*?)\], "attorneys": (.*?)(?=, "head_matter":), "head_matter": (".*?")(?=, "judges": ), "judges": \[(.*?)\]...\n', r'\1\t\2\t\3\t\4\t\5\t\6\n', result)
                count_list[i] += count
                i = i + 1
                
                result = record_id + '\t' + result
                fout.write(result)
                count_sub = i

        for j in range(count_sub): 
            print('substitution ' + str(j + 1) + ' count: ' + str(count_list[j])) 

substitution 1 count: 183149
substitution 2 count: 183149
substitution 3 count: 14
substitution 4 count: 183135
substitution 5 count: 183149
substitution 6 count: 183149
substitution 7 count: 183149
substitution 8 count: 183149
